In [91]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 500)
pd.set_option("display.min_rows", 200)
pd.set_option('display.max_colwidth', None)

pd.options.mode.chained_assignment = None



In [229]:
ddf = pd.read_csv('../data/indiana/indiana_guardianship_dockets.csv')
cdf = pd.read_csv('../data/indiana/indiana_clean_agc_filings.csv')

In [239]:
ddf.case_number.nunique()

9725

## Cleaned Dataset EDA

In [96]:
# how many neither approved or denied

missing_outcome_casenums = cdf[~cdf.guardianship_approved & ~cdf.guardianship_denied].case_number.unique().tolist()

In [97]:
missing_outcome_casenums[:10]

['89d01-2311-gu-000110',
 '89d02-2108-gu-000075',
 '49d08-2311-gu-043156',
 '49d08-2011-gu-040400',
 '02d03-1905-gu-000081',
 '49d08-2108-gu-028600',
 '59c01-2112-gu-000110',
 '02d03-2010-gu-000229',
 '49d08-1810-gu-043335',
 '53c01-1701-gu-000009']

In [98]:
print('Percent missing disposition:', round(len(missing_outcome_casenums)/cdf.shape[0]*100, 1), '%')

Percent missing disposition: 32.3 %


In [230]:
def clean_text(df, colname):
    space_replace_list = ['\n', '\/']
    for sp in space_replace_list:
        df[colname] = df[colname].str.replace(sp, ' ', regex=True)
    df[colname] = df[colname].str.replace("[^A-Za-z ]", '', regex=True)
    df[colname] = df[colname].str.replace("\s+", ' ', regex=True)
    df[colname] = df[colname].str.replace(" co\s+", ' co', regex=True)
    return df[colname].str.strip().str.lower()

ddf['combined_docket_event_description'] = ddf['docket_event_description'].fillna('') + ' ' + ddf['docket_event_comment'].fillna('')
ddf['combined_docket_event_description'] = clean_text(ddf, 'combined_docket_event_description')

vardf = pd.read_csv('~/Downloads/indiana_2024_variables.csv')
vardf['value'] = clean_text(vardf, 'value')
var_dicts = vardf.to_dict('records')
var_cols = vardf[~vardf.value.isnull()].varname.unique().tolist()

## Reading in Variables

Notes:
* Remove "Order Granting" because too many possibilities
* 

In [ ]:
"((temporary|permanent) )?guardianship ((is|was) )?granted", "appoint.*guardian", "guardian appoint", "granted ((permanent|temporary) )?guardianship" (excluding "not|reject|deny|denied")

In [231]:
check_values = lambda val: (ddf.docket_event_description.str.lower() == val) | ddf.docket_event_comment.str.lower().str.contains(val)
check_partial_values = lambda val: (ddf.combined_docket_event_description.str.lower().str.contains(val, na=False))

for var in var_dicts:
    if var['varname'] and (var['value'] == var['value']):
        try:
            val = var['value'].lower().strip()
            ddf[var['varname']] = check_partial_values(val)
            print(var['varname'], var['value'], ddf[var['varname']].sum())
        except:
            print('ERROR', var['value'])

summed_cases = ddf.groupby('case_number')[var_cols].sum().reset_index()
cdf_merge = cdf.merge(summed_cases, on='case_number', how='left')

keyword_cols = lambda val: [c for c in cdf_merge.columns if c.startswith(val) and val + '_' not in c]

medical_ physicians report filed 3960
medical1 dr evaluation 1
medical2 psycho evaluation 1
accounting_1  order approving biennial accounting 1918
p1 verified petition for appointment of adult guardian efiled 1
p2 petition for appointment of a guardian conservator 0
p3 petition for appointment of coguardians over person and estate 85
p4 petition to establish guardianship 45728
p5 petitioning court for permanent guardianship 1
pcon conservatorship order for hearing 1
p_emerg petition for emergency temporary guardianship filed 2179
p_emerg1 motion for interim order 3
p_emerg2 petition for emergency appointment of temporary guardian 43
p_emerg3 request for initial guardianship hearing 1
temp_order appointing temporary guardian over person estate 3
temp_order1 order granting temporary guardianship 4448
temp_order2 acceptance of appointment as temporary guardian 0
temp_order3 order appointing temporary guardian of the person and conservator 1
temp_order4 letters of guardianship issued tempo

In [234]:
cdf_merge['gc_denied'] = cdf_merge[keyword_cols('long_denied')].sum(axis='columns') > 0
print('Permanent guardianships denied:', cdf_merge['gc_denied'].sum())

cdf_merge['perm_gc_approved'] = cdf_merge[keyword_cols('long')].sum(axis='columns') > 0
print('Permanent guardianships granted:', cdf_merge['perm_gc_approved'].sum())

cdf_merge['temp_gc_approved'] = cdf_merge[keyword_cols('temp_order')].sum(axis='columns') > 0
print('Temporary guardianships granted:', cdf_merge['temp_gc_approved'].sum())

cdf_merge['limited_gc_approved'] = cdf_merge[keyword_cols('limited_order')].sum(axis='columns') > 0
print('Limited guardianships granted:', cdf_merge['limited_gc_approved'].sum())

cdf_merge['gc_dismissed'] = cdf_merge[keyword_cols('dismiss')].sum(axis='columns') > 0
print('Petition guardianship dismissed:', cdf_merge['gc_dismissed'].sum())
 
print('Overlap between granted/denied:', (cdf_merge['guardianship_denied'] & cdf_merge['perm_gc_approved']).sum())
print('Overlap between temp and perm AGC orders:', (cdf_merge['temp_gc_approved'] & cdf_merge['perm_gc_approved']).sum())

# how many neither approved or denied
outcome_cols = ['perm_gc_approved', 'temp_gc_approved', 'limited_gc_approved', 'gc_denied', 'gc_dismissed']
missing_df = cdf_merge.copy()
for c in outcome_cols:
    missing_df = missing_df[~missing_df[c]]

print('Missing disposition count:', missing_df.shape[0])

Permanent guardianships denied: 91
Permanent guardianships granted: 7391
Temporary guardianships granted: 2028
Limited guardianships granted: 17
Petition guardianship dismissed: 2532
Overlap between granted/denied: 32
Overlap between temp and perm AGC orders: 1382
Missing disposition count: 1133


In [ ]:
cdf_merge.to_csv('../data/indiana/indiana_clean_guardianship.csv', index=False)

In [237]:
missing_df.sample(10)

,c2dp_case_key,fips,state,county,case_number,last_collected,clean_case_type,is_closed,date_filed,date_disposed,disposition,ward_gender,ward_race,case_status,guardianship_status,guardianship_approved,guardianship_denied,date_approved,date_ended,last_docket,last_docket_date,petitioner_name,guardian_name,respondent_name,petitioner_attorney,guardian_attorney,respondent_attorney,gal_or_casa,medical_,medical1,medical2,accounting_1,p1,p2,p3,p4,p5,pcon,p_emerg,p_emerg1,p_emerg2,p_emerg3,temp_order,temp_order1,temp_order2,temp_order3,temp_order4,temp_order5,temp_order6,temp_order7,limited_order1:,limited_order2,limited_order3,Dismiss_temp,long1,long2,long3,long5,long6,long7,long8,long9,long10,long11,long12,long13,long14,long15,long16,longcons1,long_denied,dismiss1,dismiss2,dismiss3,dismiss4,dismiss5,dismiss6,dismiss7,dismiss8,dismiss9,death,death1,death2,death3,death4,death5,death6,gc_denied,perm_gc_approved,temp_gc_approved,limited_gc_approved,gc_dismissed
3144,indiana___46D01-2004-GU-000040,18091,indiana,laporte,46d01-2004-gu-000040,2023-02-17 05:33:44.000,gu - guardianship - guardian: adult,True,2020-04-28,NaN,NaN,male,white,case_closed,not_approved,False,False,NaN,NaN,Automated ENotice Issued to Parties,05/06/2020,"Cicero, Patrick",NaN,"Cicero, Alexander",Amber Lee Poff,NaN,NaN,False,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False
1370,indiana___02D03-1706-GU-000120,18003,indiana,allen,02d03-1706-gu-000120,2023-02-15 22:13:29.000,gu - guardianship - guardian: adult,True,2017-06-21,NaN,NaN,missing,missing,case_closed,not_approved,False,False,NaN,NaN,Automated ENotice Issued to Parties,07/21/2017,"Louraine, Nancy J",NaN,"Ehle, Patricia J",Ronald Dean Felger,NaN,NaN,False,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False
3804,indiana___52D01-1806-GU-000003,18103,indiana,miami,52d01-1806-gu-000003,2023-02-16 08:42:38.000,gu - guardianship - guardian: adult,True,2018-06-27,NaN,NaN,female,white,case_closed,not_approved,False,False,NaN,NaN,Case File Destroyed,03/18/2020,"Avila, Melissa",NaN,"Blount, Margaret Elizabeth",William Kelly Leeman,NaN,NaN,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False
7025,indiana___45D11-1811-GU-000287,18089,indiana,lake,45d11-1811-gu-000287,2023-02-16 13:40:36.000,gu - guardianship - guardian: adult,True,2018-11-19,NaN,NaN,female,black,case_closed,not_approved,False,False,NaN,NaN,Scanned/Microfilmed,03/03/2021,"BLACKMON, CASAUNDRA",NaN,"GRIGGS, JONNA L",Pro Se,NaN,NaN,False,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False
4706,indiana___49D08-2307-GU-029882,18097,indiana,marion,49d08-2307-gu-029882,2024-01-27 14:03:13.000,gu - guardianship - guardian: adult,False,2023-07-28,NaN,NaN,female,white,case_open,case_pending,False,False,NaN,NaN,Automated Paper Notice Issued to Parties,10/05/2023,"WOODS, DEBRA",NaN,"GWINN, ELIZABETH ANNE",Adam Michael Goss,NaN,NaN,False,1,0,0,0,0,0,0,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False
3625,indiana___53C01-2305-GU-000079,18105,indiana,monroe,53c01-2305-gu-000079,2023-11-24 14:01:08.000,gu - guardianship - guardian: adult,False,2023-05-25,NaN,NaN,female,missing,case_open,case_pending,False,False,NaN,NaN,Automated ENotice Issued to Parties,08/01/2023,"Linne, Melissa E",NaN,"Linne, Savannah R",NaN,NaN,NaN,False,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False
628,indiana___45C01-1708-GU-000147,18089,indiana,lake,45c01-1708-gu-000147,2023-02-15 23:24:09.000,gu - guardianship - guardian: adult,False,20

In [235]:
ddf[ddf.case_number == '10C04-1603-GU-000038'.lower()]

,c2dp_case_id,fips,case_key,state,county,case_number,raw_case_type,clean_case_type,date_filed,date_closed,date_disposed,disposition,raw_case_status,clean_case_status,is_closed,docket_eventkey,docket_casekey,docket_baseeventtype,docket_baseeventtypekey,docket_eventtype,docket_event_date,docket_event_description,docket_event_comment,combined_docket_event_description,medical_,medical1,medical2,accounting_1,p1,p2,p3,p4,p5,pcon,p_emerg,p_emerg1,p_emerg2,p_emerg3,temp_order,temp_order1,temp_order2,temp_order3,temp_order4,temp_order5,temp_order6,temp_order7,limited_order1:,limited_order2,limited_order3,Dismiss_temp,long1,long2,long3,long5,long6,long7,long8,long9,long10,long11,long12,long13,long14,long15,long16,longcons1,long_denied,dismiss1,dismiss2,dismiss3,dismiss4,dismiss5,dismiss6,dismiss7,dismiss8,dismiss9,death,death1,death2,death3,death4,death5,death6
261891,10C04-1603-GU-000038,18019,indiana___10C04-1603-GU-000038,indiana,clark,10c04-1603-gu-000038,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2016-03-15,2016-05-18,NaN,NaN,decided,disposed/closed,True,297429486,22650770,C,C,QCSNEW,03/15/2016,Case Opened as a New Filing,NaN,case opened as a new filing,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
261892,10C04-1603-GU-000038,18019,indiana___10C04-1603-GU-000038,indiana,clark,10c04-1603-gu-000038,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2016-03-15,2016-05-18,NaN,NaN,decided,disposed/closed,True,297429682,22650770,C,C,APP,03/15/2016,Appearance Filed,GR - Filed electronically,appearance filed gr filed electronically,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
261893,10C04-1603-GU-000038,18019,indiana___10C04-1603-GU-000038,indiana,clark,10c04-1603-gu-000038,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2016-03-15,2016-05-18,NaN,NaN,decided,disposed/closed,True,297429684,22650770,C,C,APP,03/15/2016,Appearance Filed,GR - Filed electronically,appearance filed gr filed electronically,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
261894,10C04-1603-GU-000038,18019,indiana___10C04-1603-GU-000038,indiana,clark,10c04-1603-gu-000038,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2016-03-15,2016-05-18,NaN,NaN,decided,disposed/closed,True,297429685,22650770,C,ORGFD,PEG,03/15/2016,Petition to Establish Guardianship Filed,GR - Filed electronically,petition to establish guardianship filed gr filed electronically,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
261895,10C04-1603-GU-000038,18019,indiana___10C04-1603-GU-000038,indiana,clark,10c04-1603-gu-000038,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2016-03-15,2016-05-18,NaN,NaN,decided,disposed/closed,True,297430329,22650770,C,MOT,MHRG,03/15/2016,Motion

In [162]:
ddf[check_partial_values('gal casa')].combined_docket_event_description.unique()

array(['petition to appoint gal casa filed petition for appointment of guardian ad litem',
       'gal casa appointed gal william krowl', 'gal casa appointed cc',
       'gal casa appointed joseph shikany',
       'petition to appoint gal casa filed petition to appoint guardian ad litem',
       'petition to appoint gal casa filed petition for gal',
       'gal casa appointed party to notify',
       'gal casa appointed damian gosheff',
       'gal casa appointed thomas allen is appointed gal party to notify',
       'gal casa appointed thomas allen',
       'gal casa appointed gal r jeff dodson',
       'gal casa appointed terri francis',
       'petition to appoint gal casa filed order appointing guardian ad litem',
       'petition to appoint gal casa filed petition for appointment of gal',
       'petition to appoint gal casa filed petition to appoint gal',
       'private gal casa appointed marc a morrison rbe',
       'gal casa appointed melissa cunnyngham is appointed gal',
    

## Docket EDA

In [5]:
ddf.columns

Index(['c2dp_case_id', 'fips', 'case_key', 'state', 'county', 'case_number',
       'raw_case_type', 'clean_case_type', 'date_filed', 'date_closed',
       'date_disposed', 'disposition', 'raw_case_status', 'clean_case_status',
       'is_closed', 'docket_eventkey', 'docket_casekey',
       'docket_baseeventtype', 'docket_baseeventtypekey', 'docket_eventtype',
       'docket_event_date', 'docket_event_description'],
      dtype='object')

In [6]:
NUM_OF_EXAMPLES = 10
def get_docket_entry_data(df):
    data = {}
    data['count'] = df.case_number.nunique()
    data['case_examples'] = "; ".join(df.sample(NUM_OF_EXAMPLES, replace=True).case_number.unique().tolist())
    return pd.Series(data)

frequency_table = ddf.groupby('docket_event_description').apply(get_docket_entry_data).sort_values('count', ascending=False).reset_index()
frequency_table.to_csv(f'../data/indiana/docket_keyword_search_results/all.csv')

### Comment EDA

In [ ]:
dates or numbers/symbols generally
names
----

In [34]:
ddf['clean_comment'] = ddf.docket_event_comment.str.lower()
print('Variations before:', ddf.clean_comment.nunique())
erase_list = ['----.*', '[\d\/\,\.]', '\: *(am|pm)', '\n']
for e in erase_list:
    ddf['clean_comment'] = ddf['clean_comment'].str.replace(e, ' ', regex=True)
print('Variations after cleaning:', ddf.clean_comment.nunique())

Variations before: 146739
Variations after cleaning: 73427


In [23]:
ddf[ddf.docket_event_comment.str.contains('Acceptance of Appointment as Temporary Guardian', na=False)]

,c2dp_case_id,fips,case_key,state,county,case_number,raw_case_type,clean_case_type,date_filed,date_closed,date_disposed,disposition,raw_case_status,clean_case_status,is_closed,docket_eventkey,docket_casekey,docket_baseeventtype,docket_baseeventtypekey,docket_eventtype,docket_event_date,docket_event_description,docket_event_comment,medical_,medical1,medical2,accounting_1,p1,p2,p3,p4,p5,pcon,p_emerg,p_emerg1,p_emerg2,p_emerg3,temp_order,temp_order1,temp_order2,temp_order3,temp_order4,temp_order5,temp_order6,temp_order7,limited_order1:,limited_order2,limited_order3,Dismiss_temp,long1,long2,long3,long4,long5,long6,long7,long8,long9,long10,long11,long12,long13,long14,longcons1,long_denied,dismiss1,dismiss2,dismiss3,dismiss4,dismiss5,dismiss6,dismiss7,dismiss8,dismiss9,death,death1,death2,death3,death4,death5,death6


### Keyword Searches

In [8]:
ddf.head()

,c2dp_case_id,fips,case_key,state,county,case_number,raw_case_type,clean_case_type,date_filed,date_closed,date_disposed,disposition,raw_case_status,clean_case_status,is_closed,docket_eventkey,docket_casekey,docket_baseeventtype,docket_baseeventtypekey,docket_eventtype,docket_event_date,docket_event_description
0,02D03-1708-GU-000149,18003,indiana___02D03-1708-GU-000149,indiana,allen,02d03-1708-gu-000149,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2017-08-02,2017-08-04,NaN,NaN,decided,disposed/closed,True,380563984,28090563,C,C,QCSNEW,08/02/2017,Case Opened as a New Filing
1,02D03-1708-GU-000149,18003,indiana___02D03-1708-GU-000149,indiana,allen,02d03-1708-gu-000149,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2017-08-02,2017-08-04,NaN,NaN,decided,disposed/closed,True,380564210,28090563,C,ORGFD,PEG,08/02/2017,Petition to Establish Guardianship Filed
2,02D03-1708-GU-000149,18003,indiana___02D03-1708-GU-000149,indiana,allen,02d03-1708-gu-000149,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2017-08-02,2017-08-04,NaN,NaN,decided,disposed/closed,True,380564214,28090563,C,ORGFD,PETG,08/02/2017,Petition for Emergency/Temporary Guardianship ...
3,02D03-1708-GU-000149,18003,indiana___02D03-1708-GU-000149,indiana,allen,02d03-1708-gu-000149,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2017-08-02,2017-08-04,NaN,NaN,decided,disposed/closed,True,380564217,28090563,C,OTHER,NF,08/02/2017,Notice Filed
4,02D03-1708-GU-000149,18003,indiana___02D03-1708-GU-000149,indiana,allen,02d03-1708-gu-000149,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2017-08-02,2017-08-04,NaN,NaN,decided,disposed/closed,True,380564220,28090563,C,C,ADMOAF,08/02/2017,Oath Filed


Looking for "granted", "approved"

In [9]:
def search_keyword(wrds: list[str], persist=False): 
    kwdf = frequency_table
    for wd in wrds:
        kwdf = kwdf[kwdf.docket_event_description.str.lower().str.contains(wd, regex=True)]

    if persist:
        filename = "__and__".join([wrd.replace("(","").replace(")", "").replace("|", "_or_") for wrd in wrds])
        print(filename)
        kwdf.to_csv(f'../data/indiana/docket_keyword_search_results/{filename}.csv')
    return kwdf

In [10]:
search_keyword

<function __main__.search_keyword(wrds: list[str], persist=False)>

In [11]:
search_keyword(['(grant|approv|accept)'], persist=True)

grant_or_approv_or_accept


,docket_event_description,count,case_examples
8,Order Granting Petition to Establish Guardianship,4443,53c01-1811-gu-000148; 82d04-1610-gu-000543; 18...
25,Order Granting Motion to Terminate Guardianship,1546,49d08-1802-gu-004469; 49d08-2010-gu-034923; 49...
32,Order Granting Motion to Withdraw Appearance,1241,49d08-1711-gu-043504; 49d08-1705-gu-020694; 41...
36,Order Granting,1196,27d03-1608-gu-000058; 45d05-1905-gu-000147; 17...
37,Order Granting Temporary Guardianship,1101,48c01-1903-gu-000146; 49d08-1810-gu-039870; 10...
43,Order Granting Motion for Continuance,866,41d01-1708-gu-000131; 45d02-1905-gu-000146; 76...
45,Order Approving,819,49d08-1611-gu-040144; 42c01-1603-gu-000024; 49...
53,Order Granting Motion for Allowance of Fees,583,49d08-1810-gu-041476; 47c01-1609-gu-000065; 03...
57,Order Granting Motion for Fee Waiver,548,27d03-1804-gu-000030; 73d01-1612-gu-000070; 49...
61,Order Approving Biennial Accounting,468,52d02-1901-gu-000001; 49d08-1611-gu-041562; 53...


In [12]:
search_keyword(['(deny|reject)'], persist=True)

deny_or_reject


,docket_event_description,count,case_examples
117,Order Denying,95,49d08-1608-gu-029944; 49d08-1908-gu-033436; 49...
148,Order Denying Petition to Establish Guardianship,47,73c01-1801-gu-000002; 41d01-1909-gu-000171; 22...
175,Order Denying Motion for Continuance,32,49d08-1703-gu-011792; 79c01-2010-gu-000083; 45...
270,Order Denying Motion to Correct Error,6,49d08-2009-gu-033922; 49d08-1706-gu-023009; 49...
297,Order Denying Motion for Fee Waiver,5,52d01-1604-gu-000003; 28c01-1609-gu-000026; 79...
328,Order Denying Motion to Dismiss,4,49d08-1603-gu-008392; 49d08-1906-gu-022867; 49...
338,Order Denying Motion for Appointment,3,77c01-1908-gu-000027; 82d04-1602-gu-000083; 89...
345,Order Denying Motion to Consolidate,3,49d08-2007-gu-024495; 49d08-1711-gu-043747; 49...
354,Order Denying Motion to Terminate Guardianship,3,31c01-1611-gu-000055; 45d02-1807-gu-000138; 45...
382,Order Denying Motion to Reconsider,2,49d08-1605-gu-018488; 49d08-1906-gu-025473


In [13]:
search_keyword(['(guardian|conservator)'], persist=True)

guardian_or_conservator


,docket_event_description,count,case_examples
5,Petition to Establish Guardianship Filed,5152,29d03-1612-gu-000226; 10c04-1707-gu-000113; 02...
7,Letters of Guardianship Issued,5062,49d08-1811-gu-044988; 49d08-1612-gu-043009; 27...
8,Order Granting Petition to Establish Guardianship,4443,53c01-1811-gu-000148; 82d04-1610-gu-000543; 18...
9,Hearing on Petition to Establish Guardianship,4293,02d03-1810-gu-000278; 49d08-2011-gu-042052; 49...
17,Guardianship Information Sheet Filed,2102,02d03-2003-gu-000066; 17d02-2005-gu-000013; 49...
25,Order Granting Motion to Terminate Guardianship,1546,49d08-1802-gu-004469; 49d08-2010-gu-034923; 49...
33,Petition for Emergency/Temporary Guardianship ...,1238,45c01-1807-gu-000145; 49d08-2001-gu-001024; 49...
37,Order Granting Temporary Guardianship,1101,48c01-1903-gu-000146; 49d08-1810-gu-039870; 10...
42,Instruction Executed - Executor/Personal Rep/ ...,866,82d04-1612-gu-000682; 49d08-1607-gu-026735; 10...
46,Motion to Terminate Guardianship Filed,694,82d04-1610-gu-000571; 82d04-1704-gu-000216; 41...


In [14]:
search_keyword(['(guardian|conservator|gal|casa)','(grant|accept|deny|reject|approv|order)'], persist=True)

guardian_or_conservator_or_gal_or_casa__and__grant_or_accept_or_deny_or_reject_or_approv_or_order


,docket_event_description,count,case_examples
8,Order Granting Petition to Establish Guardianship,4443,53c01-1811-gu-000148; 82d04-1610-gu-000543; 18...
25,Order Granting Motion to Terminate Guardianship,1546,49d08-1802-gu-004469; 49d08-2010-gu-034923; 49...
37,Order Granting Temporary Guardianship,1101,48c01-1903-gu-000146; 49d08-1810-gu-039870; 10...
123,Order Granting Extension of Guardianship,84,49d08-1904-gu-013870; 49d08-1907-gu-029342; 49...
148,Order Denying Petition to Establish Guardianship,47,73c01-1801-gu-000002; 41d01-1909-gu-000171; 22...
238,Order Granting Motion to Remove Guardian,10,10c01-1612-gu-000191; 49d08-1804-gu-013870; 41...
354,Order Denying Motion to Terminate Guardianship,3,31c01-1611-gu-000055; 45d02-1807-gu-000138; 45...
388,Order Denying Motion to Remove Guardian,2,27d03-1706-gu-000051; 49d08-2008-gu-028430
435,Order Denying Extension of Guardianship,1,43d01-1809-gu-000071


In [15]:
search_keyword(['petition'], persist=True)

petition


,docket_event_description,count,case_examples
5,Petition to Establish Guardianship Filed,5152,29d03-1612-gu-000226; 10c04-1707-gu-000113; 02...
8,Order Granting Petition to Establish Guardianship,4443,53c01-1811-gu-000148; 82d04-1610-gu-000543; 18...
9,Hearing on Petition to Establish Guardianship,4293,02d03-1810-gu-000278; 49d08-2011-gu-042052; 49...
23,Petition Filed,1746,02d03-1812-gu-000331; 02d03-1804-gu-000086; 49...
33,Petition for Emergency/Temporary Guardianship ...,1238,45c01-1807-gu-000145; 49d08-2001-gu-001024; 49...
48,Hearing on Petition for Emergency Temporary Gu...,656,49d08-2009-gu-033413; 37c01-1608-gu-000646; 02...
59,Petition to Appoint GAL/CASA Filed,522,02d03-1604-gu-000096; 02d03-1704-gu-000060; 02...
148,Order Denying Petition to Establish Guardianship,47,73c01-1801-gu-000002; 41d01-1909-gu-000171; 22...
159,Petition for Appointment of Personal Represent...,37,41d01-1909-gu-000175; 02d03-1804-gu-000106; 41...
170,Petition for Extension of Temporary Guardianship,32,45c01-1809-gu-000215; 49d08-2001-gu-000430; 49...


In [16]:
search_keyword(['temporary'], persist=True)

temporary


,docket_event_description,count,case_examples
33,Petition for Emergency/Temporary Guardianship ...,1238,45c01-1807-gu-000145; 49d08-2001-gu-001024; 49...
37,Order Granting Temporary Guardianship,1101,48c01-1903-gu-000146; 49d08-1810-gu-039870; 10...
48,Hearing on Petition for Emergency Temporary Gu...,656,49d08-2009-gu-033413; 37c01-1608-gu-000646; 02...
170,Petition for Extension of Temporary Guardianship,32,45c01-1809-gu-000215; 49d08-2001-gu-000430; 49...
222,Temporary Appearance Filed,13,10c01-1712-gu-000187; 45c01-1802-gu-000026; 49...
299,Motion for Temporary Restraining Order Filed,5,79c01-2010-gu-000083; 49d08-2006-gu-021479; 37...
494,Order Granting Motion for Temporary Restrainin...,1,49d08-2006-gu-021479
502,Hearing on Motion for Temporary Restraining Order,1,49d08-2006-gu-021479


In [17]:
search_keyword(['(death|decease|dead|demise|died|funeral)'], persist=True)

death_or_decease_or_dead_or_demise_or_died_or_funeral


,docket_event_description,count,case_examples


In [18]:
frequency_table[frequency_table.docket_event_description.str.contains('Notice Filed')]

,docket_event_description,count,case_examples
14,Notice Filed,2283,02d03-2009-gu-000199; 49d08-1603-gu-009994; 46...
73,Waiver of Notice Filed,323,45d05-1808-gu-000177; 41d01-2008-gu-000131; 59...
311,Proof of Publication of Notice Filed,4,18c01-1811-gu-000140; 20d01-1609-gu-000123; 46...
350,Lis Pendens Notice Filed,3,80c01-1908-gu-000022; 49d08-1812-gu-048993


In [19]:
search_keyword(['dismiss'], persist=True)

dismiss


,docket_event_description,count,case_examples
68,Motion to Dismiss Filed,402,49d08-1707-gu-029295; 49d06-1904-gu-017473; 49...
74,Order of Dismissal,315,49d08-1606-gu-021955; 49d08-1910-gu-044622; 49...
89,Order Granting Motion to Dismiss,216,49d08-1606-gu-020109; 49d08-1701-gu-004265; 49...
142,Hearing on Motion for 41(E) Dismissal,52,34d04-1812-gu-000098; 49d08-2009-gu-031626; 49...
158,Hearing on Motion for 41E Dismissal,38,18c01-1606-gu-000060; 03c01-1709-gu-005029; 02...
221,Order of 41E Dismissal,13,71d05-1608-gu-000164; 55d01-1610-gu-000100; 27...
250,Dismissed,8,48c03-1906-gu-000348; 48c06-1710-gu-000486; 18...
252,Notice of Dismissal Filed,8,49d08-1610-gu-036977; 49d08-2009-gu-030536; 49...
266,Hearing on Motion to Dismiss,6,73d02-1604-gu-000023; 49d08-1712-gu-046127; 71...
269,Stipulation of Dismissal Filed,6,49d04-1605-gu-015747; 49d08-1712-gu-044650; 41...


### Missing Outcome Frequency Table